Written by [Gameli Ladzekpo](mailto:gameli.Ladzekpo@gmail.com) (Twitter/IG: @gamladz)

For [AI Core](theaicore.com)

In [13]:
# Start with imports 

import json 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as expected_conditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
from time import sleep, time
import random
import re
import subprocess, os
import urllib.request
import sys
import boto3
from botocore.exceptions import ClientError
from selenium import webdriver 
import time
import requests

In [14]:
# Set up AWS functions
s3 = boto3.resource("s3").Bucket("ikea-dataset")
json.load_s3 = lambda f: json.load(s3.Object(key=f).get()["Body"])
json.dump_s3 = lambda obj, f: s3.Object(key=f).put(Body=json.dumps(obj))
client = boto3.client('s3', region_name='us-east-2')

# Example usage

# json.dump_s3(data, "key")  saves json to s3://bucket/key
# data = json.load_s3("key")  read json from s3://bucket/key

In [15]:
# open_chrome function
def open_chrome(port=9220, on_mac=True):
    my_env = os.environ.copy()
    if on_mac:
        subprocess.Popen(['open', '-a', "Google Chrome", '--args', f'--remote-debugging-port={port}', 'http://www.example.com'], env=my_env)
    else:
        subprocess.Popen(f'google-chrome --remote-debugging-port={port} --user-data-dir=bots'.split(), env=my_env)

class Bot():
    def __init__(self, port_no = 9220, headless = False, verbose = False):
        print('initialising bot')

        open_chrome()

        options = Options()
        options.add_argument("--no-sandbox")	# without this, the chrome webdriver can't start (SECURITY RISK)
        options.add_experimental_option(f"debuggerAddress", f"127.0.0.1:{port_no}")	# attach to the same port that you're running chrome on
        if headless:
            options.add_argument("--headless") # headless option allows scraper to run in the background
        #options.add_argument("--window-size=1920x1080")
        self.driver = webdriver.Chrome('chrome_driver/chromedriver')			# create webdriver
        self.verbose = verbose

    def click_btn(self, text):
        if self.verbose: print(f'clicking {text} btn')
        element_types = ['button', 'div', 'input', 'a', 'label']
        
        for element_type in element_types:
            btns = self.driver.find_elements_by_xpath(f'//{element_type}')
            # for btn in btns:
            #   print(btn.text)

            # SEARCH BY TEXT
            try:
                btn = [b for b in btns if b.text.lower() == text.lower()][0]
                btn.click()
                return
            except IndexError:
                pass

            # SEARCH BY VALUE ATTRIBUTE IF NOT YET FOUND
            try:
                btn = self.driver.find_elements_by_xpath(f'//{element_type}[@value="{text}"]')[0]
                btn.click()
                return
            except:
                continue

        raise ValueError(f'button containing "{text}" not found')

    def _search(self, query, _type='search', placeholder=None):
        sleep(1)
        s = self.driver.find_elements_by_xpath(f'//input[@type="{_type}"]')
        print(s)
        if placeholder:
            s = [i for i in s if i.get_attribute('placeholder').lower() == placeholder.lower()][0]
        else:
            s = s[0]
        s.send_keys(query) 

    def toggle_verbose(self):
        self.verbose = not self.verbose

In [17]:
if __name__ == '__main__':
    bot = Bot()
    
    data_dict = {
        
    }

    searches = {
        "plant pots":[], 
        "bin":[],
        "cookware":[],
        "wardrobes":[], 
        "tables":[],
        "cabinets":[],
        "clothing racks":[],
        "chair":[]
    }
    
    for search in searches:
        bot.driver.get(f'https://www.ikea.com/gb/en/search/products/?q={search}')
        data_dict[search] = {}

        # Get full grid by trying to scroll and press show more button
        while True:
            try:
                bot.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(5)
                show_more_button = bot.driver.find_elements_by_xpath('//*[@class="show-more__button button button--secondary button--small"]')[0]
                show_more_button.click()
            except IndexError:
                break


        # Go into the main grid and find the link for each result
        results = bot.driver.find_elements_by_xpath('//*[@id="search-results"]/div/a')  
        print (f'found {len(results)}) results for search "{search}" ')


        results = [r.get_attribute('href') for r in results]

        for i, result in enumerate(results):

            # Skip promotional links, adverts or non-product links in grid
            if '/p/' not in result:
                continue 

            bot.driver.get(result)
            result = result.split('/')[-2]

            # Set up dict for each products result
            results_dict = {
                "id": [],
                "prod_name": [],
                "prod_price": [],
                "prod_desc": [],
                "prod_dimensions": [],
                "packaging": [],
                "prod_details": [],
                "sustainability": [],
                "images":[],
                "materials":[]
            }
            

            results_dict['id'] = hash(result)


            # Extract Product Description
            prod_name = bot.driver.find_element_by_xpath('//*[@class="range-revamp-header-section__title--big"]')
            prod_name = prod_name.get_attribute('innerHTML')              
            results_dict['prod_name'] = prod_name
                        

            prod_price = bot.driver.find_element_by_xpath('//*[@class="range-revamp-price__integer"]')
            prod_price = prod_price.get_attribute('innerHTML')
            results_dict['prod_price'] = prod_price


            prod_desc = bot.driver.find_element_by_xpath('//*[@class="range-revamp-header-section__description-text"]')
            prod_desc = prod_desc.get_attribute('innerHTML')
            results_dict['prod_desc'] = prod_desc


            os.makedirs(f'data/{search}/{result}', exist_ok=True)

            # First get the image - product dimensions image
            prod_dims_images = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-product-dimensions-content__images"]//img')
            prod_dims_images = [i.get_attribute('src') for i in prod_dims_images]

            for image in prod_dims_images:
                # Loop through each image and save to disk

                for idx, img_url in enumerate(images):

                    # Create filename for each with ID and get the file extension
                    filename = f'dims-{result}-{idx}'
                    file_ext = img_url.split('.')[-1] 
                    file_ext = file_ext[ 0 : 3 ]
                    # Write file locally
                    filepath = f'data/{search}/{result}/{filename}.{file_ext}'
                    try:
                        urllib.request.urlretrieve(img_url, filepath)
                    except ConnectionResetError:
                        print ('Connection Error')
                    else:
                        img_data = requests.get(img_url).content
                        with open(filepath, 'wb') as handler:
                            handler.write(img_data)                         
                    finally:
                        pass
                    # Write image to AWS
                    with open(filepath, 'rb') as data:
                        s3.upload_fileobj(data, filepath)
                    # remove comment to stop storing
                    # os.remove(filepath)

            # for image in prod_dims_images:
                
            
            # Get Product Details Packaging
            prod_packaging = bot.driver.find_elements_by_xpath('//*[@id="SEC_product-details-packaging"]//span')
            prod_packaging = [item.get_attribute('innerHTML') for item in prod_packaging]
            prod_packaging = [x for x in prod_packaging if "</span>" not in x]
            results_dict['packaging'] = prod_packaging

            # Second get all product dimensions
            prod_dims_name = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-product-dimensions__list-container"]//dt')
            prod_dims_name = [item.get_attribute('innerHTML').split(":")[0] for item in prod_dims_name]
        
            
            prod_dims_measure = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-product-dimensions__list-container"]//dd')
            prod_dims_measure = [item.get_attribute('innerHTML') for item in prod_dims_measure]
            dims = dict(zip(prod_dims_name, prod_dims_measure))
            results_dict['prod_dimensions'] = dims


            # Cycle through images 
            images = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-media-grid__media-container"]//img') 
            images = [i.get_attribute('src') for i in images]

            # Product Details
            product_details = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-product-details__paragraph"]')
            product_details = [paragraph.get_attribute('innerHTML') for paragraph in product_details]
            product_details = " ".join(product_details)
            results_dict['prod_details'] = product_details

            # Materials
            product_details_materials = bot.driver.find_elements_by_xpath('//*[@id="SEC_product-details-material-and-care"]//span')
            product_details_materials = [paragraph.get_attribute('innerHTML') for paragraph in product_details_materials]
            product_details_materials = ",".join(product_details_materials)
            results_dict['materials'] = product_details_materials

            # Sustainability
            product_details_sustain = bot.driver.find_elements_by_xpath('//*[@id="SEC_product-details-sustainability-and-environment"]//span')
            product_details_sustain = [paragraph.get_attribute('innerHTML') for paragraph in product_details_sustain]
            product_details_sustain = ",".join(product_details_sustain)
            results_dict['sustainability'] = product_details_sustain

            

            # Get reviews
            
            for image in images:
                # Loop through each image and save to disk

                for idx, img_url in enumerate(images):

                    # Create filename for each with ID and get the file extension
                    filename = f'{result}-{idx}'
                    file_ext = img_url.split('.')[-1] 
                    file_ext = file_ext[ 0 : 3 ]
                    # Write file locally
                    filepath = f'data/{search}/{result}/{filename}.{file_ext}'
                    try:
                        urllib.request.urlretrieve(img_url, filepath)
                    except ConnectionResetError:
                        print ('Connection Error')
                    else:
                        img_data = requests.get(img_url).content
                        with open(filepath, 'wb') as handler:
                            handler.write(img_data)                         
                    finally:
                        pass
                    # Write to cloud
                    with open(filepath, 'rb') as data:
                        s3.upload_fileobj(data, filepath)
                    results_dict['images'].append(filepath)
                    # remove comment to remove file locally
                    # os.remove(filepath)
                    
            # Remove duplicate images
            results_dict['images'] = [i for j, i in enumerate(results_dict['images']) if i not in results_dict['images'][:j]] 
            # Wrte dict to local
            data_dict[search][i] = results_dict 
            with open('data_json.txt', 'w') as json_file:
                json.dump(data_dict, json_file)            
            json.dump_s3(data_dict, "data_json")



        


initialising bot
found 202) results for search "plant pots" 


NameError: name 'test_dict' is not defined

In [55]:
import pandas as pd


# with open('data_json.txt', 'r') as file:
#     data = file.read().replace('\n', '')[0:-1]

# f = open("data_json.txt", "r")

df = pd.read_json('data_json.txt')
print(df.from_dict(f))

df.read_json({(i,j): f[i][j] 
                           for i in f.keys() 
                           for j in f[i].keys()},
                       orient='index')

# print(f)
# df = pd.DataFrame(data=f)
# print(df)






                                                   0
0  {"plant pots": {"0": {"id": 0, "prod_name": "M...


AttributeError: 'DataFrame' object has no attribute 'read_json'